# Импорт

In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c485218544397da64a1e1b68ef0fffbb974d741f64b2b957da93f56615bf8487
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
# при работе через Google Colab
# прописать путь до всех файлов py
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Учёба/8 семестр/диплом/Проект/VKR/Блокноты')

Mounted at /content/drive


In [3]:
import AnalysisFunc
import BaseFuncs

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from datetime import datetime
from math import sqrt

In [5]:
directory_name = "Учет контрольных"

# Конвертация

Колонки:



In [ ]:
# открыть через виндовс
file_name = BaseFuncs.OpenFileNameWindows()
xls_array = BaseFuncs.GetDataFrameListFromFileRaw(file_name)

In [6]:
# открыть через колаб
file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/Данные_ПиОА_2023_1семестр.xlsx"
# file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/трек HARD.xlsx"
xls_array = BaseFuncs.GetDataFrameListFromFileRaw(file_name)

In [ ]:
# конвертирование данных от ИОТ
temp_row = ""
for sheet in xls_array:
    for name, group in sheet.groupby(["Название РМУП"]):
        if "Предмет контроля" not in group.columns:
            continue
        name_column_test = ""
        if "Контрольная работа" in set(list(group["Предмет контроля"])):
            name_column_test = "Контрольная работа"
        else:
            name_column_test = "Работа на учебной встрече"
        results = pd.DataFrame(columns=["ФИО", "Пол", "Баллы", "Не сдал(-а)"])

        name_discipline = group.head(1)["Название РМУП"].values[0]
        try:
            if (group["Название встречи"].str.contains('Контрольная ').sum()==0 and name_column_test!="Контрольная работа"):
                continue
            print(name_discipline, end=":")
            print(name_column_test, set(list(group["Предмет контроля"])))

            # max_scores_before_test = {}

            percent = 0
            count_presence = summ_presence = summ_goals_btest = count_goals = count_meetings = 0

            goals_list = []

            startTime = datetime.now()
            # номер контрольной
            test_number = 1
            # # количество действий с кр (баллы и результаты до)
            for index, row in group.iterrows ():

                if (index%(len(group)//10)==0):
                    print(f"{percent}%. Записей в итоге: {len(results)}")
                    percent+=10

                if temp_row == "" or temp_row != row["ФИО студента"]:
                    test_number = 1
                    count_presence = summ_presence = summ_goals_btest = count_goals = count_meetings = 0

                    temp_row = row["ФИО студента"]

                    results.loc[-1] = [temp_row, BaseFuncs.GetGender(temp_row), row["Итог ТУ"], BaseFuncs.GetInfoLose(row, "Итог ТУ")] + [""]*(len(results.columns)-4)
                    results.index = results.index + 1  # shifting index

                if ("Контрольная " in row["Название встречи"] or "Контрольная работа" == row["Предмет контроля"]):
                    meeting_name = f"Контрольная работа {test_number}"

                    if (f"Контрольная работа {test_number}" not in list(results.columns)):
                        if (f"Посещение до {meeting_name}" not in list(results.columns)):
                            results[f"Посещение до {meeting_name}"] = 0.0
                            results[f"Баллы до {meeting_name}"] = 0.0
                            results[f"Количество баллов до {meeting_name}"] = 0.0
                            results[f"Корень дисперсии баллов до {meeting_name}"] = 0.0
                            # results[f"Процент баллов до {meeting_name}"] = 0.0
                        results[meeting_name] = 0.0
                    if (row["Предмет контроля"] == name_column_test):

                        test_score = float(row["Оценка за предметы контроля"]) if row["Оценка за предметы контроля"]!="" else 0
                        results.loc[((results['ФИО'] == temp_row)), f"Контрольная работа {test_number}"] = test_score

                        if count_presence == 0:
                            continue
                        results.loc[((results['ФИО'] == temp_row)), f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                        results.loc[((results['ФИО'] == temp_row)), f"Баллы до {meeting_name}"] = summ_goals_btest
                        results.loc[((results['ФИО'] == temp_row)), f"Количество баллов до {meeting_name}"] = count_goals
                        results.loc[((results['ФИО'] == temp_row)), f"Корень дисперсии баллов до {meeting_name}"] = sqrt(np.var(goals_list))

                        count_presence = summ_presence = summ_goals_btest = count_goals = count_meetings = 0
                        goals_list = []
                        test_number+=1


                elif (row["Предмет контроля"] != "Посещение"):
                    count_meetings += 1
                    if not pd.isna(row["Оценка за предметы контроля"]):
                        goal = float(row["Оценка за предметы контроля"])
                        goals_list.append(goal)
                        summ_goals_btest += goal
                        if goal>0.0:
                            count_goals += 1
                    else:
                        goals_list.append(0)
                else:
                    count_presence += 1
                    if row["Оценка за предметы контроля"]=="П":
                        summ_presence += 1


            results = results.fillna(0)
            results = results.replace("", 0)
            results = results.drop(results.columns[-5:], axis= 1)

            print(f"Время выполнения: {datetime.now()-startTime}")
            print("Отдельное сохранение...")

            BaseFuncs.SaveFileCSV(path = f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}", filename = f"{name_discipline}", results = results)
            print("Сохранено")
        except Exception as e:
            print(e)



Алгебра:Работа на учебной встрече {'Посещение', 'Работа на учебной встрече'}
0%. Записей в итоге: 94
10%. Записей в итоге: 189
20%. Записей в итоге: 285
30%. Записей в итоге: 380
40%. Записей в итоге: 475
50%. Записей в итоге: 570
60%. Записей в итоге: 666
70%. Записей в итоге: 760
80%. Записей в итоге: 856
90%. Записей в итоге: 951
Время выполнения: 0:00:15.524449
Отдельное сохранение...
Сохранено
Программирование и основы алгоритмизации 1:Контрольная работа {'Посещение', 'Работа на учебной встрече', 'Контрольная работа'}
0%. Записей в итоге: 84
10%. Записей в итоге: 180
20%. Записей в итоге: 276
30%. Записей в итоге: 372
40%. Записей в итоге: 468
50%. Записей в итоге: 564
60%. Записей в итоге: 661
70%. Записей в итоге: 756
80%. Записей в итоге: 852
90%. Записей в итоге: 948
Время выполнения: 0:00:14.884025
Отдельное сохранение...
Сохранено
Математический анализ:Контрольная работа {'Контрольная работа', 'Письменный ответ', 'Посещение', 'Работа на учебной встрече', 'Практическое задани

In [7]:
# очистка названий колонок и замена на первую строку в данны от Modeus
for sheet in xls_array:
    sheet.columns = sheet.loc[0, :].values.flatten().tolist()
    sheet.drop(index=[0], inplace=True)
    sheet = sheet.reset_index(drop=True)

In [8]:
results = pd.DataFrame(columns=["ФИО", "Команды", "Направление", "Пол", "Баллы", "Не сдал(-а)"])
print(results.columns)

for sheet in xls_array:
    print(f"Записей: {len(sheet)}")
    startTime = datetime.now()
    percent = 0

    # try:
    # max_scores_before_test = {}
    count_presence = summ_presence = count_kontr = 0
    summ_goals_btest = count_goals = count_meetings = summ_goals_result = 0

    for index, row in sheet.iterrows ():
        if (index%(len(sheet)//10)==0):
            print(f"{percent}%. Записей в итоге: {len(results)}")
            percent+=10
        results.loc[-1] = [row["Обучающийся"], row["Учебные команды"], row["Направление подготовки"], BaseFuncs.GetGender(row["Обучающийся"]), row["Итог текущ."], BaseFuncs.GetInfoLose(row, "Итог текущ.")] + [""]*(len(results.columns)-6)
        results.index = results.index + 1

        count_presence = summ_presence = count_kontr = 0
        summ_goals_btest = count_goals = count_meetings = summ_goals_result = 0

        goals_list = []
        for i in range(5, len(sheet.columns.to_list())):
            if sheet.columns.to_list()[i]!="Контрольная работа":
                if sheet.columns[i]=="Присутствие":
                    count_presence+=1
                    if (row[i]=="П"):
                        summ_presence+=1
                elif sheet.columns[i]=="Работа на учебной встрече":
                    count_meetings += 1
                    if not pd.isna(row[i]):
                        goal = (float(row[i]) if "." in str(row[i]) else float(str(row[i]).replace(",", ".")))
                        goals_list.append(goal)
                        summ_goals_btest += goal
                        if goal>0.0:
                            count_goals += 1
                    else:
                        goals_list.append(0)
            else:
                count_kontr+=1
                meeting_name = f"Контрольная работа {count_kontr}"

                if (f"Посещение до {meeting_name}" not in list(results.columns)):
                    results[f"Посещение до {meeting_name}"] = 0
                    results[f"Баллы до {meeting_name}"] = 0
                    results[f"Количество баллов до {meeting_name}"] = 0.0
                    results[f"Корень дисперсии баллов до {meeting_name}"] = 0.0
                    # results[f"Процент баллов до {meeting_name}"] = 0
                    results[meeting_name] = 0

                results.loc[results.index[-1], f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                results.loc[results.index[-1], f"Баллы до {meeting_name}"] = summ_goals_btest
                results.loc[results.index[-1], f"Количество баллов до {meeting_name}"] = count_goals
                results.loc[results.index[-1], f"Корень дисперсии баллов до {meeting_name}"] = sqrt(np.var(goals_list))


                test_score = float(row[i]) if row[i]!="" else 0
                results.loc[results.index[-1], meeting_name]  = test_score

                summ_goals_result += summ_goals_btest + test_score


                count_presence = summ_presence = summ_goals_btest = count_goals = 0

    results = results.fillna(0)

    print(f"Время выполнения: {datetime.now()-startTime}")
    print("Отдельное сохранение...")
    BaseFuncs.SaveFileCSV(path = f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}", filename = "Программирование и основы алгоритмизации тест", results = results)
    print("Сохранено")
    # except Exception as e:
    #     print("Ошибка:", e)

Index(['ФИО', 'Команды', 'Направление', 'Пол', 'Баллы', 'Не сдал(-а)'], dtype='object')
Записей: 961
0%. Записей в итоге: 95
10%. Записей в итоге: 191
20%. Записей в итоге: 287
30%. Записей в итоге: 383
40%. Записей в итоге: 479
50%. Записей в итоге: 575
60%. Записей в итоге: 671
70%. Записей в итоге: 767
80%. Записей в итоге: 863
90%. Записей в итоге: 959
Время выполнения: 0:00:04.865482
Отдельное сохранение...
Сохранено
